# YouTube Playlist Duration Calculator

### 3 Ways

#### 1.   requests
> ##### pros : quick
> ##### cons : loads only one page so used for small playlists

#### 2.   selenium automation
* Chrome
> ##### pros : supports larger playlists
> ##### cons : too much slower for larger playlists
* Brave
> ##### pros : 
> ##### cons : 
* PhantomJS
> ##### pros : 
> ##### cons : 

#### 3.   youtube API
> ##### pros : faster
> ##### cons : get all videos data that are not available

In [75]:
URL = "https://www.youtube.com/playlist?list=PLZDMziPRHWIrkfzKIXUOrir_gwD0cD9f8" # TEST
# URL = "https://www.youtube.com/playlist?list=PLR-r0edywujc9bj3vpNjqIDg-WbWj3vyz" # Normal Playlist
# URL = "https://youtube.com/playlist?list=PLFsQleAWXsj_4yDeebiIADdH5FMayBiJo" # 300
# URL = "https://youtube.com/playlist?list=PL4lEESSgxM_5O81EvKCmBIm_JT5Q7JeaI" # 1000
# URL = "https://youtube.com/playlist?list=PLG9huCBGQQq33BxkisPUvWx8mUNOVyg4w" # 5000

## requests

In [76]:
import requests
import re

page = requests.get(URL)

pattern = "\\{\"accessibility\":\\{\"accessibilityData\":\\{[^}]*\\}\\},\"simpleText\":\"[^\"]*\"\\}"
dl = list(set(re.findall(pattern, page.text)))
pa1 = "\\d*:\\d*:\\d*"
pa2 = '"."\\d*:\\d*"'
durations = []
for i in dl:
	pal1 = re.findall(pa1, i)
	if len(pal1) != 0:
		durations.append(pal1[0][:])
for i in dl:
	pal2 = re.findall(pa2, i)
	if len(pal2) != 0:
		durations.append(pal2[0][3:-1])
print(len(durations),sorted(durations))

8 ['0:04', '0:34', '10:00:05', '11:19', '168:00:07', '1:08:23', '200:00:01', '4:13']


In [77]:
def sol1(arr): 
 days = hours = mins = secs = 0 
 for duration in arr: 
  try: 
   secs += int(duration[-2:]) 
   mins += int(duration[-5:-3]) 
   hours += int(duration[:-6]) 
  except ValueError: 
   continue 
 mins += (secs//60) 
 secs %= 60 
 hours += (mins//60) 
 mins %= 60 
 days = hours//24 
 hours %= 24 
 return f'{days}day,{hours}Hr,{mins}min,{secs}sec'

def sol2(arr): 
	days = hours = mins = secs = 0  
	for duration in arr: 
		try:  
			secs += int(duration[-2:])  
			mins += int(duration[-5:-3])  
			hours += int(duration[:-6])  
		except ValueError:  
			continue  
	mins += (secs//60)  
	secs %= 60  
	hours += (mins//60)  
	mins %= 60  
	days = hours//24  
	hours %= 24  
	return (f'{days}day,' if days > 0 else '') + (f'{hours}Hr,' if hours > 0 else '') + (f'{mins}min,' if mins > 0 else '') + (f'{secs}sec')
 
print(sol1(durations))
print(sol2(durations))

15day,19Hr,24min,46sec
15day,19Hr,24min,46sec


In [78]:
for i in range(len(durations)):
	if durations[i].count(":") == 1:
		durations[i] = "0:"+durations[i]
print(len(durations),sorted(durations))

import datetime
mysum = datetime.timedelta()
for i in durations:
    (h, m, s) = i.split(':')
    d = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))
    mysum += d
print(mysum)

from datetime import timedelta
def to_td(h):
    ho, mi, se = h.split(':')
    return timedelta(hours=int(ho), minutes=int(mi), seconds=int(se))
print(str(sum(map(to_td,durations), timedelta())))

totalSecs = 0
for tm in durations:
    timeParts = [int(s) for s in tm.split(':')]
    totalSecs += (timeParts[0] * 60 + timeParts[1]) * 60 + timeParts[2]
totalSecs, sec = divmod(totalSecs, 60)
hr, min = divmod(totalSecs, 60)
print(hr,min,sec)

total = 0
for line in durations:
    h, m, s = map(int, line.split(":"))
    total += 3600*h + 60*m + s
print(int(total / 3600),int(total / 60 % 60),int( total % 60))

import functools
sum_fnc=lambda ttt:(lambda a:'%02d:%02d:%02d' % (divmod(divmod(a,60)[0],60)+(divmod(a,60)[1],)))((lambda a:functools.reduce(lambda x,y:x+y[0]*3600+y[1]*60+y[2],a,0))((lambda a:[list(map(int,i.split()[-1].split(':'))) for i in a])(ttt)))
print(sum_fnc(durations))

8 ['0:0:04', '0:0:34', '0:11:19', '0:4:13', '10:00:05', '168:00:07', '1:08:23', '200:00:01']
15 days, 19:24:46
15 days, 19:24:46
379 24 46
379 24 46
379:24:46


## selenium
### Chrome

In [79]:
!pip install selenium
!apt-get update # to update linux os to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubunt

In [80]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup as soup
import re

In [81]:
def get_html():
    innerHTML = driver.execute_script("return document.body.innerHTML")
    page_soup = soup(innerHTML, 'html.parser')
    return page_soup

In [82]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument("--incognito")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [83]:
ser = Service("chromedriver")
driver = webdriver.Chrome(service=ser,options=chrome_options)
driver.get(URL)
pat1 = r'"numVideosText":\{"runs":\[\{"text":"([,0-9]*)"\},\{"text":" videos"\}]\}'
ns = re.findall(pat1, driver.execute_script("return document.body.innerHTML"))
times_scroll_down = int((int(ns[0].replace(",",""))/100) + 2)
for i in range(times_scroll_down):
    elm = driver.find_element(By.TAG_NAME,'html')
    elm.send_keys(Keys.END)
    # import time
    # time.sleep(1)
page_soup = get_html()
driver.quit() # stop the driver
tc = page_soup.findAll('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'})
pat2 = r'\n  ([0-9]+(:[0-9]+)+)\n'
lt = [re.findall(pat2, str(i))[0][0] for i in tc]
print(int(ns[0]),len(lt),lt)

8 8 ['0:04', '0:34', '4:13', '11:19', '1:08:23', '10:00:05', '168:00:07', '200:00:01']


In [84]:
def sol1(arr): 
 days = hours = mins = secs = 0 
 for duration in arr: 
  try: 
   secs += int(duration[-2:]) 
   mins += int(duration[-5:-3]) 
   hours += int(duration[:-6]) 
  except ValueError: 
   continue 
 mins += (secs//60) 
 secs %= 60 
 hours += (mins//60) 
 mins %= 60 
 days = hours//24 
 hours %= 24 
 return f'{days}day,{hours}Hr,{mins}min,{secs}sec'

def sol2(arr): 
	days = hours = mins = secs = 0  
	for duration in arr: 
		try:  
			secs += int(duration[-2:])  
			mins += int(duration[-5:-3])  
			hours += int(duration[:-6])  
		except ValueError:  
			continue  
	mins += (secs//60)  
	secs %= 60  
	hours += (mins//60)  
	mins %= 60  
	days = hours//24  
	hours %= 24  
	return (f'{days}day,' if days > 0 else '') + (f'{hours}Hr,' if hours > 0 else '') + (f'{mins}min,' if mins > 0 else '') + (f'{secs}sec')
 
print(sol1(lt))
print(sol2(lt))

15day,19Hr,24min,46sec
15day,19Hr,24min,46sec


In [85]:
for i in range(len(lt)):
    if lt[i].count(":") == 1:
        lt[i] = "0:"+lt[i]
print(len(lt),lt)

import datetime
mysum = datetime.timedelta()
for i in lt:
    (h, m, s) = i.split(':')
    d = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))
    mysum += d
print(mysum)

from datetime import timedelta
def to_td(h):
    ho, mi, se = h.split(':')
    return timedelta(hours=int(ho), minutes=int(mi), seconds=int(se))
print(str(sum(map(to_td,lt), timedelta())))

totalSecs = 0
for tm in lt:
    timeParts = [int(s) for s in tm.split(':')]
    totalSecs += (timeParts[0] * 60 + timeParts[1]) * 60 + timeParts[2]
totalSecs, sec = divmod(totalSecs, 60)
hr, min = divmod(totalSecs, 60)
print(hr,min,sec)

total = 0
for line in lt:
    h, m, s = map(int, line.split(":"))
    total += 3600*h + 60*m + s
print(int(total / 3600),int(total / 60 % 60),int( total % 60))

import functools
sum_fnc=lambda ttt:(lambda a:'%02d:%02d:%02d' % (divmod(divmod(a,60)[0],60)+(divmod(a,60)[1],)))((lambda a:functools.reduce(lambda x,y:x+y[0]*3600+y[1]*60+y[2],a,0))((lambda a:[list(map(int,i.split()[-1].split(':'))) for i in a])(ttt)))
print(sum_fnc(lt))

8 ['0:0:04', '0:0:34', '0:4:13', '0:11:19', '1:08:23', '10:00:05', '168:00:07', '200:00:01']
15 days, 19:24:46
15 days, 19:24:46
379 24 46
379 24 46
379:24:46


## API

1. Create a [YouTube Data v3 API key](https://developers.google.com/youtube/registering_an_application)
2. Replace the strings with your own API key and playlist ID.

In [86]:
!pip install isodate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
from datetime import timedelta
import datetime
import isodate
import json
import requests
import re

In [88]:
def duration_format(a):
    pattern = '(([0-9]+) day[s]?, )?([0-9]+):([0-9]+):([0-9]+)'
    ree = re.findall(pattern, str(a))

    if ree[0][-4] != '':
        days = int(ree[0][-4])
    else : days = 0

    hours = int(ree[0][-3])
    mins = int(ree[0][-2])
    secs = int(ree[0][-1])

    return((f'{days}day,' if days > 0 else '') + (f'{hours}Hr,' if hours > 0 else '') + (f'{mins}min,' if mins > 0 else '') + (f'{secs}sec'))

In [89]:
# replace with your api
yt_api = os.getenv('YT_API_KEY') # take from github secrets 
# replace with your playlist_id
playlist_id = URL[URL.index("=")+1:]

URL1 = 'https://www.googleapis.com/youtube/v3/playlistItems?part=contentDetails&maxResults=50&fields=items/contentDetails/videoId,nextPageToken&key={}&playlistId={}&pageToken='.format(yt_api, playlist_id)
URL2 = 'https://www.googleapis.com/youtube/v3/videos?&part=contentDetails&key={}&id={}&fields=items/contentDetails/duration'.format(yt_api, "{}")

In [90]:
next_page = ''
cnt = 0
a = timedelta(0)

while True:

    vid_list = [] 

    results = json.loads(requests.get(URL1 + next_page).text)
    
    for x in results['items']:
        vid_list.append(x['contentDetails']['videoId'])
        
    cnt += len(vid_list)

    url_list = ','.join(vid_list)

    op = json.loads(requests.get(URL2.format(url_list)).text)

    for x in op['items']:
        a += isodate.parse_duration(x['contentDetails']['duration'])

    if 'nextPageToken' in results:
        next_page = results['nextPageToken']
    else:
        print(f'Total length of playlist : {str(a)} : {duration_format(a)}',
                f'\nNo of videos : {str(cnt)} : {int(cnt)}', 
                f'\nAverage length of video : {str(a/cnt)} : {duration_format(a/cnt)}',
                f'\nAt 1.25x : {str(a/1.25)} : {duration_format(a/1.25)}', 
                f'\nAt 1.50x : {str(a/1.5)} : {duration_format(a/1.5)}', 
                f'\nAt 1.75x : {str(a/1.75)} : {duration_format(a/1.75)}', 
                f'\nAt 2.00x : {str(a/2)} : {duration_format(a/2)}')
        break

Total length of playlist : 15 days, 19:24:46 : 15day,19Hr,24min,46sec 
No of videos : 8 : 8 
Average length of video : 1 day, 23:25:35.750000 : 1day,23Hr,25min,35sec 
At 1.25x : 12 days, 15:31:48.800000 : 12day,15Hr,31min,48sec 
At 1.50x : 10 days, 12:56:30.666667 : 10day,12Hr,56min,30sec 
At 1.75x : 9 days, 0:48:26.285714 : 9day,48min,26sec 
At 2.00x : 7 days, 21:42:23 : 7day,21Hr,42min,23sec
